In [19]:
import ccxt
from datetime import datetime, timedelta, time as tm
import pandas as pd
import numpy as np
import warnings
import asyncio
import time
from IPython.display import clear_output
import os

In [ ]:
trading = [['XTZ','ALGO',2.9], ['VET','IOTA',2], ['ETC','BCH',2.9], ['BAT','QTUM',2.3]]

In [21]:
def read_trading_df(trading):
    if 'trading_df.csv' in os.listdir():
        trading_df = pd.read_csv('trading_df.csv',index_col = 0, header = 0)
        
    else:
        trading_df = pd.DataFrame([],columns = ['ticker','coef'])
        for i in range(len(trading)):
            lst = trading[i]

            trading_df = trading_df.append(pd.DataFrame({'ticker':[lst[0]+'USDT',lst[1]+'USDT'],'coef':lst[2]},index=[i]*2))
        trading_df[['o','bid','ask','bid_change','ask_change','position_price','position_type','vol','delta']]=0
        trading_df['current_time'] = None
        counter = {}
        for ticker in trading_df.ticker:
            counter[ticker]=0
        total_vol =[0]
    return trading_df

trading_df = read_trading_df(trading)
trading_df

,ticker,coef,o,bid,ask,bid_change,ask_change,position_price,position_type,vol,delta,current_time
0,XTZUSDT,2.9,4.37900,4.38300,4.384000,0.0913,0.1142,4.08600,Buy,2.447381,0,3
0,ALGOUSDT,2.9,1.36440,1.36630,1.366800,0.1393,0.1759,1.35780,Sell,7.364855,0,3
1,VETUSDT,2.0,0.07983,0.07995,0.079960,0.1503,0.1628,0.07782,Buy,128.501671,0,3
1,IOTAUSDT,2.0,1.06410,1.06600,1.066300,0.1786,0.2067,1.03650,Sell,9.647853,0,3
2,ETCUSDT,2.9,35.38500,35.47000,35.478000,0.2402,0.2628,34.39900,Buy,0.290706,0,3
2,BCHUSDT,2.9,437.01000,437.59000,437.668169,0.1327,0.1506,424.14000,Sell,0.023577,0,3
3,BATUSDT,2.3,1.05750,1.05880,1.058900,0.1229,0.1324,1.03760,Sell,9.637625,0,3
3,QTUMUSDT,2.3,9.17800,9.19000,9.191000,0.1307,0.1416,8.89800,Buy,1.123848,0,3


### Continue

In [4]:
def prepearing(trading_df):
    tables ={}
    time_lst =[]
#     now = datetime.fromtimestamp(req_client.get_servertime()/1e3)-timedelta(minutes=15)
#     end_time = datetime(now.year, now.month, now.day, now.time().hour, now.time().minute//15*15)
    
    
    for index in set(trading_df.index.values):
        tickers = trading_df.loc[trading_df.index==index, 'ticker'].values
        base = pd.DataFrame([])
         
        for ticker in tickers:
            candles = candles = exchange.fetch_ohlcv(ticker, '15m')
            ticker_df = pd.DataFrame([[datetime.fromtimestamp(x[0]/1e3), x[4]] for x in candles], columns = ['time', 'c'])
            
            if 'time' not in base.columns.values:
                base['time'] = ticker_df.time.values
                base[ticker] = ticker_df.c.values
            else:
                base[ticker] = None
                for time, c in ticker_df.values:
                    base.loc[base.time==time, ticker] = c
            
            base.sort_values('time',ascending=True,inplace = True,ignore_index=True)    
            lst=[0]
            for index in base.index[1:]:
                lst.append(round((float(base[ticker][index])/float(base[ticker][index-1])-1)*100, 4))
            base[ticker+'_delta'] = lst
                
        base['delta'] = base[tickers[0]+'_delta']-base[tickers[1]+'_delta']
        tables[tickers[0]+'-'+tickers[1]] = base[:-1]
    return tables

def calculate_vol( price1, price2):
    return 10/price1, 10/price2   

def calc_avg_price(orderbook, limit):
    price=np.array([])
    vol=np.array([])
    index=0
    while np.sum(vol*price)<limit and index < len(orderbook)-1:
        price = np.append(price, orderbook[index][0])
        vol = np.append(vol, orderbook[index][1])
        index+=1
    value = np.sum(price*vol)/np.sum(vol)
    return value

In [5]:
def callback_candle(candles, ticker, tickers):

    open_price = candles[-1][1]
    time_now = datetime.fromtimestamp(candles[-1][0]/1e3)

    prepearing_base = prepearing_df[tickers[0]+'-'+tickers[1]]
    prev_time = np.datetime64(datetime.fromtimestamp(candles[-2][0]/1e3))
    if prev_time not in prepearing_base.time.values:
        dic_to_add={'time':prev_time,
                   ticker: candles[-2][4],
                   ticker+'_delta': round((candles[-2][4]/prepearing_base[ticker].values[-1] -1)*100 ,4)}
        prepearing_base = prepearing_base.append(dic_to_add, ignore_index=True)
    elif np.isnan(prepearing_base[ticker].values[-1]):
        prepearing_base.loc[prepearing_base.time==prev_time, [ticker, 
                                                              ticker+'_delta',]] = [candles[-2][4],
                                                                                    round((candles[-2][4]/prepearing_base[ticker].values[-2] -1)*100 ,4)]
        prepearing_base.loc[prepearing_base.time==prev_time, 'delta'] = prepearing_base.loc[prepearing_base.time==prev_time,tickers[0]+'_delta']-\
                                                                        prepearing_base.loc[prepearing_base.time==prev_time,tickers[1]+'_delta']

    prepearing_df[tickers[0]+'-'+tickers[1]] = prepearing_base
    trading_df.loc[trading_df.ticker==ticker, ['o', 'current_time']] = [open_price,
                                                                        time_now.time().minute//15]
    
    
def callback_orderbook(orderbook, ticker):

    bid = calc_avg_price(orderbook['bids'], 10)
    ask = calc_avg_price(orderbook['asks'], 10)

    open_price = trading_df.loc[trading_df.ticker==ticker, 'o'].values[0]

    trading_df.loc[trading_df.ticker==ticker, ['bid',
                                               'ask',
                                               'bid_change',
                                               'ask_change']] = [bid,
                                                                 ask,
                                                                 round((bid/open_price-1) * 100,4),
                                                                 round((ask/open_price-1) * 100,4)]


In [6]:
def trade(i, tickers):
    base = trading_df.loc[trading_df.index==i]
    prepearing_base = prepearing_df[tickers[0]+'-'+tickers[1]]

    delta_1 = prepearing_base.append({'delta':float(base.bid_change.values[0])-float(base.ask_change.values[1])},
                                      ignore_index=True).delta.astype(float).values
    delta_2 = prepearing_base.append({'delta':float(base.ask_change.values[0])-float(base.bid_change.values[1])},
                                      ignore_index=True).delta.astype(float).values
#     base.loc[:,'delta'] = [round(float(np.sum(delta_1)-delta_1[-400:].mean()),1),
#                            round(float(np.sum(delta_2)-delta_2[-400:].mean()),1)]

    if np.sum(base.vol* base.position_price) <1:

        #open SHOT-LONG
        if np.sum(delta_1)-delta_1[-400:].mean()  > base.coef.values[0]\
        and all(base.current_time.values):

            lot_1,lot_2 = calculate_vol(base.bid.values[0], base.ask.values[1])

            #sell_func


            base.loc[base.ticker == tickers[0], ['position_price',
                                                'position_type',
                                                'vol']]=[base.bid.values[0],
                                                         'Sell',
                                                         lot_1]
            base.loc[base.ticker == tickers[1], ['position_price',
                                                'position_type',
                                                'vol']]=[base.ask.values[1],
                                                         'Buy',
                                                         lot_2]

            total_vol[0]+= (base.bid.values[0]*lot_1+base.ask.values[1]*lot_2)*2
            #applying changes    
            trading_df.loc[trading_df.index==base.index[0], 
                           trading_df.columns] = base.values

        #open LONG-SHORT     

        elif np.sum(delta_2)-delta_2[-400:].mean() < -base.coef.values[0]\
        and all(base.current_time.values):

            lot_1,lot_2 = calculate_vol(base.ask.values[0], base.bid.values[1])

            # buy_func

            base.loc[base.ticker == tickers[0], ['position_price',
                                                'position_type',
                                                'vol']]=[base.ask.values[0],
                                                         'Buy',
                                                         lot_1]
            base.loc[base.ticker == tickers[1], ['position_price',
                                                'position_type',
                                                'vol']]=[base.bid.values[1],
                                                         'Sell',
                                                         lot_2]
            total_vol[0]+= (base.ask.values[0]*lot_1+base.bid.values[1]*lot_2)*2
            #applying changes    
            trading_df.loc[trading_df.index==base.index[0], 
                           trading_df.columns] = base.values


    else:

        #closing LONG-SHORT
        if base.position_type.values[0]=='Buy':


            result_1 = ((base.bid.values[0] -base.position_price.values[0])/base.position_price.values[0])*100
            result_2 = ((base.position_price.values[1] - base.ask.values[1])/base.position_price.values[1])*100

            if np.sum(delta_1)-delta_1[-400:].mean()>0\
            and all(base.current_time.values):

                # sell_func

                counter[base.ticker.values[0]]+= (result_1-0.04)/100*base.position_price.values[0]*base.vol.values[0]
                counter[base.ticker.values[1]]+= (result_2-0.04)/100*base.position_price.values[1]*base.vol.values[1]


                base.loc[:, ['position_price','position_type','vol']]=0


                #applying changes    
                trading_df.loc[trading_df.index==base.index[0], 
                               trading_df.columns] = base.values




        #closing SHORT-LONG    
        elif base.position_type.values[0]=='Sell':

            result_1 = ((base.position_price.values[0] - base.ask.values[0])/base.position_price.values[0])*100
            result_2 = ((base.bid.values[1] -base.position_price.values[1])/base.position_price.values[1])*100

            if np.sum(delta_2)-delta_2[-400:].mean() < 0 \
            and all(base.current_time.values):

                # buy_func

                counter[base.ticker.values[0]]+= (result_1-0.04)/100*base.position_price.values[0]*base.vol.values[0]
                counter[base.ticker.values[1]]+= (result_2-0.04)/100*base.position_price.values[1]*base.vol.values[1]

                base.loc[:, ['position_price','position_type','vol']]=0

                #applying changes    
                trading_df.loc[trading_df.index==base.index[0], 
                               trading_df.columns] = base.values

    summ =sum([(row[3]-row[7])*row[9] if row[8]=='Buy' \
               else (row[7]-row[4])*row[9] if row[8]=='Sell'  \
               else 0 for row in trading_df.values])
    clear_output(wait = True)
    print(trading_df[['ticker','coef','bid_change',
                             'ask_change','position_price',
                             'position_type','vol','delta']], '\n',
          f'Operation profit: {round(summ,5)} \n'
          f' Fixed profit: {round(sum(counter.values()), 2)} \n',
          f'Total_vol: {round(total_vol[0],2)}')

In [7]:
api_key = 'your api key'
secret_key = 'your secret key'

exchange = ccxt.binance({
    'apiKey': api_key,
    'secret': secret_key,
    'enableRateLimit': True, # required https://github.com/ccxt/ccxt/wiki/Manual#rate-limit
    'options': {
        'defaultType': 'future',
    },
})

In [13]:
prepearing_df = prepearing(trading_df)

In [18]:
while True:
    for i in set(trading_df.index.values):
        tickers = trading_df.loc[trading_df.index==i,'ticker'].values
        if datetime.now().minute%15 == 0:
            try:
                for ticker in tickers:
                    print(1)
                    callback_candle(exchange.fetch_ohlcv(ticker,'15m', limit=2), ticker, tickers)
                    print(2)
                    callback_orderbook(exchange.fetch_order_book(ticker, 10), ticker)
                    print(3)
                trade(i, tickers)
                trading_df.to_csv('trading_df.csv')
            except:
                continue

    time.sleep(5)

KeyboardInterrupt: 